   # PIK3R1: Effects of Mutation

<b>Standard imports for playing with and plotting data frames.</b>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
#import altair as alt

#alt.renderers.enable('notebook')

<b>Import CPTAC data</b>

In [2]:
import CPTAC

Loading Dictionary...
Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter CPTAC.embargo() to open the webpage for more details.


In [3]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

### List of proteins that interact with PIK3R1 (according to Uniprot)

In [4]:
protList = ['ABI1', 'ABL2', 'AR', 'CBL', 'EGFR', 'ERBB2', 'ERBB3', 'ESR1', 'FASLG', 'FGFR1', 'RRAS', 'NRAS', 'PTEN', 'IRS1', 'IRS2', 'KIT', 'HRAS', 'IRS1', 'PIK3R1', 'PIK3CA', 'PIK3CD' 'RPS6KB1', 'AKT1', 'MRAS']

### Proteome abundance of proteins that interact with PIK3R1

In [5]:
gene = 'PIK3R1'
sigList = [];

pcutoff = 0.05/len(protList)
#pcutoff = 1

genedf = somatic_mutations[gene].to_frame()
for protein in protList:
    if protein in proteomics.columns:
        proteindf = proteomics[protein].to_frame()
        proteindfName = protein + " protein"
        proteindf.columns = [proteindfName]
        cross = genedf.add(proteindf, fill_value=0).dropna(axis=0)
        mutated = cross.loc[cross[gene] == 1.0]
        wt = cross.loc[cross[gene] == 0.0]
        ttest = scipy.stats.ttest_ind(mutated[proteindfName], wt[proteindfName])
        if ttest[1] <= pcutoff:
            sigList.append(protein)
            print("Test for " + protein + ": ")
            print(ttest)

KeyError: 'PIK3R1'

### List of significantly affected proteins

In [7]:
print(sigList)

['IRS2', 'PIK3CA']


### Phosphoproteome abundance of interacting proteins

In [8]:
phosProtList = ['ABI1', 'ABL2', 'EGFR', 'ERBB', 'ESR', 'FASLG', 'FGFR', 'RAS', 'PTEN', 'KIT', 'HRAS', 'IRS', 'PIK3R', 'PIK3CA', 'PIK3CD' 'RPS6KB', 'AKT', 'MRAS']

In [11]:
gene = 'PIK3R1'
genedf = somatic_mutations[gene].to_frame()
sigPhosResults = [];

for protein in phosProtList:
    sites = phos.filter(regex=protein)
    genedf = genedf.add(sites, fill_value=0)

mutated = genedf.loc[genedf[gene] == 1.0]
wt = genedf.loc[genedf[gene] == 0.0]

pcutoff = 0.05 / len(genedf.columns)
print(pcutoff)
#pcutoff = 1

for loc in genedf.columns:
    if not loc == gene:
        print(loc)
        mutsitedf = mutated[[gene, loc]].dropna()
        wtsitedf = wt[[gene, loc]].dropna()
        ttest = scipy.stats.ttest_ind(mutsitedf[loc], wtsitedf[loc])
        if(ttest[1] <= pcutoff):
            sigPhosResults.append(loc)
            print('Results for ' + loc + ': ')
            print(ttest)
 

0.0003448275862068966
ABI1-S183
ABI1-S216
ABI1-S222
ABI1-S225
ABI1-S231
ABI1-S323
ABI1-T200
ABI1-T229
ABL2-S602
ABL2-S620
ABL2-S631
ABL2-S817
ABL2-S819
ABL2-S820
AKT1-S124
AKT1-S126
AKT1-S129
AKT1S1-S108
AKT1S1-S112
AKT1S1-S203
AKT1S1-S222
AKT1S1-S223
AKT1S1-S231
AKT1S1-S232
AKT1S1-T110
AKT1S1-T266
AKT2-T451
AKT3-T447
EGFR-S1039
EGFR-S1042
EGFR-S1064
EGFR-S1166
EGFR-T1041
EGFR-T693
ERBB2-S1054
ERBB2-S1078
ERBB2-S1083
ERBB2-S1107
ERBB2-S1151
ERBB2-S998
ERBB2-T1166
ERBB2-T701
ERBB3-S686
ESR1-S167
ESRP1-S237
ESRP1-S543
ESRP2-S563
ESRRA-S19
ESRRA-S22
ESRRA-S26
ESRRA-S27
ESRRA-T29
FGFR1OP-S152
FGFR1OP-S156
FGFR1OP-S160
FGFR1OP-S279
GORASP1-S220
GORASP1-S241
GORASP1-T216
GORASP1-T237
GORASP2-S214
GORASP2-T222
GORASP2-T225
GORASP2-T411
GORASP2-T415
GORASP2-T433
GPRASP2-S328
GPRASP2-S330
GPRASP2-S512
GRASP-S94
IRS1-S1005
IRS1-S1100
IRS1-S1101
IRS1-S1134
IRS1-S270
IRS1-S312
IRS1-S323
IRS1-S330
IRS1-S337
IRS1-S348
IRS1-S374
IRS1-S413
IRS1-S415
IRS1-S419
IRS1-S421
IRS1-S527
IRS1-S531
IRS1-S629
IR

### List of significantly affected phosphorylation sites

In [10]:
print(sigPhosResults)

['IRS2-S1100', 'IRS2-S731']
